In [ ]:
from kiteconnect import KiteConnect
import pandas as pd
from datetime import date
api_key="zka582z590jag8yh"

secret_key="9zdlmklim6rsakd2fkhay59hybsm5mw6"

kite = KiteConnect(api_key,secret_key)

print(kite.login_url())


In [ ]:
import datetime as dt


In [ ]:
data = kite.generate_session(api_secret=secret_key,request_token="PnLBl662QC0l317cIl7lcqJLr7CNs5ia")
kite.set_access_token(data["access_token"])


access_token=data["access_token"]
public_token= data["public_token"]
user_id= data["user_id"]


In [ ]:
data

In [ ]:
from datetime import datetime, timedelta
## Getting Historical data


NSE_data= kite.instruments(exchange="NSE")

print(NSE_data)


NFO_data= kite.instruments("NFO")


In [ ]:
fut_s={}
for i in range(len(NFO_data)):
    if((NFO_data[i]["tradingsymbol"].startswith("ICICI"))&(NFO_data[i]['instrument_type']=='FUT')):
        fut_s[NFO_data[i]["expiry"]]=NFO_data[i]["instrument_token"]
        
instrument_token=fut_s[max(fut_s.keys())]

In [ ]:
fut_s

In [ ]:
instrument_token

In [ ]:
N=20
from_date=(datetime.now() - timedelta(days=N))#.strftime('%Y-%m-%d')
to_date=datetime.today()#.strftime('%Y-%m-%d')

interval="minute"
contract={}
for i in range(len(NFO_data)):
    if((NFO_data[i]["instrument_token"]==instrument_token)):
        contract=NFO_data[i]
        print(NFO_data[i])

k=NFO_data[i]['tradingsymbol']

import itertools
["".join(x) for _, x in itertools.groupby(NFO_data[i]['tradingsymbol'], key=str.isdigit)][0]

In [ ]:
from_date,to_date

In [ ]:
df=kite.historical_data(instrument_token=instrument_token,from_date=from_date,to_date=to_date,interval=interval, continuous=0)

In [ ]:
df=pd.DataFrame(df)
df.set_index('date',inplace=True)

In [ ]:
df.tail()

In [ ]:
df.to_csv("{}.csv".format(contract['tradingsymbol']))

In [ ]:
try:
    df = pd.read_csv("{}.csv".format(contract['tradingsymbol']))
    last_index=df.date.iloc[-1]
    last_index=last_index[:-6]
    last_index
except:
     with open("{}.csv".format(contract['tradingsymbol']), 'wb') as f:
            f.close()

In [ ]:
last_index=pd.to_datetime(last_index) 

In [ ]:
last_index

In [ ]:
import time
while True :
    if ((datetime.now().hour <16) & (datetime.now().minute<30)): 
        if datetime.now().second//60 ==0:
            try:
                from_date=last_index
                to_date= datetime.now()
                next_data=kite.historical_data(instrument_token=instrument_token,from_date=from_date,to_date=to_date,interval=interval, continuous=0)
                next_dataframe=pd.DataFrame(next_data)
                next_dataframe.set_index('date',inplace=True)
                df.append(next_dataframe[1:])
            except:
                time.sleep(1)
                continue
        else:
            time.sleep(1)
    else:
        break

In [ ]:
df.to_csv("{}.csv".format(contract['tradingsymbol']))

In [ ]:
df

In [ ]:
next_data

In [ ]:
for i in range(len(NFO_data)):
    if(NFO_data[i]["tradingsymbol"] and NFO_data[i]["segment"]=="NFO-FUT"):
        Futures_data= pd.DataFrame([NFO_data[i] for i in range(len(NFO_data)) if  NFO_data[i]["segment"]=="NFO-FUT"])
        Futures_data.loc[Futures_data["tradingsymbol"].str.startswith("ICICI")]    
    
    elif(NFO_data[i]["tradingsymbol"] and NFO_data[i]["segment"]=="NFO-OPT"):
        Options_data= pd.DataFrame([NFO_data[i] for i in range(len(NFO_data)) if  NFO_data[i]["segment"]=="NFO-OPT"])


In [ ]:
kite.margins('equity')

In [ ]:

##Getting the tick data


# Initialise.
global kws
kws = WebSocket(api_key, public_token, user_id)
# Callback for tick reception.
global df
df=[]

def on_tick(tick, ws):
#    print(tick)
    for _ in tick:
        if dt.datetime.now().hour!=15 and dt.datetime.now().minute!=29:
            

            df.append(_)
           # print(_)
        else:
            kws.disconnect()
    

# Callback for successful connection.
def on_connect(ws):
	# Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
	ws.subscribe([738561])

	# Set RELIANCE to tick in `full` mode.
	ws.set_mode(ws.MODE_FULL, [738561])

# Assign the callbacks.
kws.on_tick = on_tick
kws.on_connect = on_connect

# To enable auto reconnect WebSocket connection in case of network failure
# - First param is interval between reconnection attempts in seconds.
# Callback `on_reconnect` is triggered on every reconnection attempt. (Default interval is 5 seconds)
# - Second param is maximum number of retries before the program exits triggering `on_noreconnect` calback. (Defaults to 50 attempts)
# Note that you can also enable auto reconnection	 while initialising websocket.
# Example `kws = WebSocket("your_api_key", "your_public_token", "logged_in_user_id", reconnect=True, reconnect_interval=5, reconnect_tries=50)`
kws.enable_reconnect(reconnect_interval=5, reconnect_tries=50)

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect()


## Placing an order
order_id=kite.order_place(tradingsymbol="SBIN",quantity =1, order_type="MARKET",exchange="NSE",transaction_type="BUY",product="NRML")

print(kite.orders(order_id))

In [ ]:
df

In [ ]:
df1=pd.DataFrame(df)

In [ ]:
df1.to_csv("Tick_Z.csv")